### Compression de Huffman

In [7]:
module type ImpPQ = sig
  type 'a t
  val empty : unit -> 'a t
  val is_empty : 'a t -> bool
  val add : 'a t -> 'a -> unit
  val take_min : 'a t -> 'a
end

module Q : ImpPQ = struct
  type 'a tree = E | N of 'a * 'a tree * 'a tree
  type 'a t = 'a tree ref
  let empty () = ref E
  let is_empty t = !t = E
  let add t x =
    let rec aux t x = match t with
      | E -> N(x, E, E)
      | N(r, g, d) -> if x < r then N(r, aux g x, d) else N(r, g, aux d x) in
    t := aux !t x
  let take_min t =
    let rec aux t = match t with
      | E -> failwith "take_min on empty queue"
      | N(r, g, d) -> if g = E then r, d 
                      else let x, g' = aux g in
                            x, N(r, g', d) in
    let x, t' = aux !t in
    t := t';
    x
    
end

module type ImpPQ =
  sig
    type 'a t
    val empty : unit -> 'a t
    val is_empty : 'a t -> bool
    val add : 'a t -> 'a -> unit
    val take_min : 'a t -> 'a
  end
module Q : ImpPQ

In [8]:
let text = "The Trichet-Draghi letter, also known as the letter of ECB to Italy, is a confidential correspondence by which, on 5 August 2011, the former and current ECB presidents Jean-Claude Trichet and Mario Draghi (outgoing Governor of the Bank of Italy) addressed to Italian government several requests in order to influence European support to drastic measures of economic rebalancing."

val text : string =
  "The Trichet-Draghi letter, also known as the letter of ECB to Italy, is a confidential correspondence by which, on 5 August 2011, the former and current ECB presidents Jean-Claude Trichet and Mario Draghi (outgoing Governor of the Bank of Italy) addressed to Italian government several requests in o"... (* string length 378; truncated *)

In [9]:
let text = "abcdefa"

val text : string = "abcdefa"

In [10]:
let get_frequence text = (*trouve la fréquence des caractères pour créer l'arbre*)
    let freq = Array.make 256 0 in
    for i = 0 to String.length text -1 do
        freq.(Char.code text.[i]) <- freq.(Char.code text.[i]) + 1
    done;
    freq;;

val get_frequence : string -> int array = <fun>

In [11]:
let freq = get_frequence text

val freq : int array =
  [|0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0;
    0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0;
    0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0;
    0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0;
    0; 2; 1; 1; 1; 1; 1; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0;
    0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0;
    0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0;
    0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0;
    0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0;
    0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0;
    0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0|]

In [12]:
type 'a tree = F of 'a | N of 'a tree * 'a tree

type 'a tree = F of 'a | N of 'a tree * 'a tree

In [20]:
let make_huffman_tree freq = 
    let q = Q.empty () in
    let n = ref 0 in
    for i = 0 to 255 do
        if freq.(i) > 0 then(
        incr n;
        Q.add q (freq.(i), F(Char.chr i)))
    done;
    for _ = 0 to !n-2 do
        let f1, t1 = Q.take_min q in
        let f2, t2 = Q.take_min q in
        Q.add q (f1+f2, N(t1, t2))
    done;
    snd(Q.take_min q);;

val make_huffman_tree : int array -> char tree = <fun>

In [21]:
let t = make_huffman_tree freq

val t : char tree =
  N (N (F 'f', F 'a'), N (N (F 'b', F 'c'), N (F 'd', F 'e')))

In [22]:
let make_table t = 
    let code = Array.make 256 [] in
    let rec aux path = function
        |F(c) -> code.(Char.code c) <- List.rev path
        |N(g, d) -> aux (0::path) g; aux (1::path) d in
    aux [] t;
    code

val make_table : char tree -> int list array = <fun>

In [23]:
let compress_huffman text = 
    let freq = get_frequence text in
    let t = make_huffman_tree freq in
    let table = make_table t in
    let rec aux i =
        if i = String.length text then []
        else table.(Char.code text.[i]) @ aux (i+1) in
    aux 0;;

val compress_huffman : string -> int list = <fun>

In [24]:
let code = compress_huffman text

val code : int list = [0; 1; 1; 0; 0; 1; 0; 1; 1; 1; 0; 1; 1; 1; 0; 0; 0; 1]

In [25]:
let rec decode_huffman t code = 
    let rec read_char t l = match t,l with
        |F(c), _ -> c, l
        |N(g, d), 0::q -> read_char g q
        |N(g, d), 1::q -> read_char d q
        |_ -> failwith "codage incorrect" in
    if code = [] then ""
    else let c, l = read_char t code in
    (String.make 1 c)^(decode_huffman t l)

val decode_huffman : char tree -> int list -> string = <fun>

In [26]:
decode_huffman t code

- : string = "abcdefa"